In [9]:
# %matplotlib notebook

In [1]:
%matplotlib qt

In [2]:
from __future__ import annotations

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (нужно для 3D)

from maneuvering.types import Scalar
from maneuvering.orbit.keplerian import Kep, KepTrue
from maneuvering.orbit.distance import distance_orbit
from maneuvering.maneuvers.quasi_circular.reference_orbit import trans_devs, reference_orbit
from maneuvering.maneuvers.quasi_circular.transition.coplanar import coplanar_non_intersecting, coplanar_analytical
from maneuvering.maneuvers.quasi_circular.transition.execute import execute, execute_batch
from maneuvering.utils.math_tools import normalize_angle
from maneuvering.maneuvers.maneuver import Maneuver

In [3]:
deg = math.pi / 180.0
mu = 3.9860044158e14

### Зависимость начальной и конечной ошибок от параметров задачи компланарного перехода

In [35]:
def build_surfaces_for_dw(dw_deg: float, de_vals: np.ndarray, da_vals: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Для фиксированного Δω (градусы) строит поверхности:
      init_err[j,i]  = distance(oi, ot)
      final_err[j,i] = distance(execute(oi, mans, μ), ot)
    где:
      oi = {a0, e0, w0=0, i0=0, Ω0=0},
      ot = {a0 + Δa, e0 + Δe, w=Δω, i=0, Ω=0}.
    Индексы: i — Δe, j — Δa.
    """
    # Базовые (абсолютные) значения
    a0 = 6_800_000.0  # [м]
    e0 = 0.0
    w0 = 0.0
    i0 = 0.0
    raan0 = 0.0

    dw = math.radians(dw_deg)

    init_err = np.zeros((da_vals.size, de_vals.size), dtype=np.float64)
    final_err = np.zeros_like(init_err)

    for j, da in enumerate(da_vals):
        for i, de in enumerate(de_vals):
            oi = Kep(a=a0, e=e0, w=w0, i=i0, raan=raan0)
            ot = Kep(a=a0 + float(da), e=e0 + float(de), w=dw, i=0.0, raan=0.0)

            # Начальная ошибка
            d0 = distance_orbit(oi, ot)

            # Манёвры и исполнение (истинные элементы с ν=0)
            mans = coplanar_analytical(oi, ot, mu)
            of_true = execute(
                oi=KepTrue(a=oi.a, e=oi.e, w=oi.w, i=oi.i, raan=oi.raan, nu=0.0),
                maneuvers=mans,
                mu=mu,
            )
            # Сравниваем геометрию (без ν)
            of_kep = Kep(a=of_true.a, e=of_true.e, w=of_true.w, i=of_true.i, raan=of_true.raan)
            d1 = distance_orbit(of_kep, ot)

            init_err[j, i] = d0
            final_err[j, i] = d1

    return init_err, final_err

In [39]:
de_vals = np.linspace(0.0, 0.1, 41)  # Δe ∈ [0, 0.1]
da_vals = np.linspace(0.0, 200_000.0, 41)  # Δa ∈ [0, 200 км] (м)
DE, DA = np.meshgrid(de_vals, da_vals)

dws = [0.0, 30.0, 60.0, 90.0]
fig = plt.figure(figsize=(16, 12))

# общая цветовая карта
cmap = cm.viridis

for k, dw in enumerate(dws, start=1):
    init_err, final_err = build_surfaces_for_dw(dw, de_vals, da_vals)

    # Один нормировщик на оба слоя, чтобы цвета были сопоставимы
    z_min = min(init_err.min(), final_err.min())
    z_max = max(init_err.max(), final_err.max())
    norm = Normalize(vmin=z_min, vmax=z_max)

    # Предварительно считаем цвета
    colors0 = cmap(norm(init_err))
    colors1 = cmap(norm(final_err))

    ax = fig.add_subplot(2, 2, k, projection="3d")
    ax.set_title(f"Δω = {int(dw)}°")

    # Две поверхности: начальная (почти непрозрачная), финальная (полупрозрачная)
    surf0 = ax.plot_surface(DE, DA, init_err, rstride=1, cstride=1,
                            facecolors=colors0, linewidth=0, antialiased=True, shade=False, alpha=0.95)
    surf1 = ax.plot_surface(DE, DA, final_err, rstride=1, cstride=1,
                            facecolors=colors1, linewidth=0, antialiased=True, shade=False, alpha=0.6)

    ax.set_xlabel("Δe, [1]")
    ax.set_ylabel("Δa, [m]")
    ax.set_zlabel("distance, [m]", labelpad=6)

    # Добавим colorbar, привязав к нормировщику
    mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
    mappable.set_array([])  # заглушка для colorbar
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.75, pad=0.05)
    cbar.set_label("Орбитальная ошибка, [м]")

fig.tight_layout()
plt.show()

RuntimeError: Colorbar layout of new layout engine not compatible with old engine, and a colorbar has been created.  Engine not changed.

### Зависимости орбитальных элементов от абсолютного угла, пройденного КА, с активным маневрированием

In [4]:
def plot_transition_series(oi, ot, mu):
    """
    1) Считает манёвры: mans = coplanar_analytical(oi, ot, mu)
    2) Прогоняет: states, angles_abs = execute_batch(oi, mans, step, mu)
       (step фиксируем внутри как 2π/720)
    3) Строит 6 графиков (2×3): a, e, ω; i, Ω, ν.
       Ось X — накопленный абсолютный угол u=ω+ν (радианы).
    Возвращает: fig, axes, states, angles_abs
    """
    # 1) Манёвры
    mans = coplanar_analytical(oi, ot, mu)

    # 2) Прогон (фиксированный шаг по u)
    states, angles_abs = execute_batch(oi, mans, mu)

    if not states:
        raise ValueError("Пустой список состояний.")

    # 3) Достаём ряды
    a = np.array([s.a for s in states], dtype=float)
    e = np.array([s.e for s in states], dtype=float)
    w = np.array([s.w for s in states], dtype=float)
    inc = np.array([s.i for s in states], dtype=float)
    Om = np.array([s.raan for s in states], dtype=float)
    nu = np.array([s.nu for s in states], dtype=float)
    x = np.asarray(angles_abs, dtype=float)  # уже в радианах

    # Разворачиваем углы (чтобы не было скачков через 2π)
    unwrap = lambda arr: np.unwrap(arr, discont=np.pi)
    w_u, i_u, Om_u, nu_u = map(unwrap, (w, inc, Om, nu))

    # Оформление
    plt.rcParams.update({
        "figure.constrained_layout.use": True,
        "axes.grid": True,
        "font.size": 11,
        "figure.figsize": (12, 6),
    })

    fig, axes = plt.subplots(2, 3, sharex=True)
    (ax_a, ax_e, ax_w), (ax_i, ax_Om, ax_nu) = axes

    # Верхний ряд
    ax_a.plot(x, a, lw=2)
    ax_a.set_title("Большая полуось a, [м]")

    ax_e.plot(x, e, lw=2)
    ax_e.set_title("Эксцентриситет e, [1]")

    ax_w.plot(x, w_u, lw=2)
    ax_w.set_title("Аргумент перицентра ω, [рад]")

    # Нижний ряд
    ax_i.plot(x, i_u, lw=2)
    ax_i.set_title("Наклонение i, [рад]")

    ax_Om.plot(x, Om_u, lw=2)
    ax_Om.set_title("Долгота восходящего узла Ω, [рад]")

    ax_nu.plot(x, nu_u, lw=2)
    ax_nu.set_title("Истинная аномалия ν, [рад]")

    # Общая ось X
    axes[1, 1].set_xlabel("Накопленный угол u = ω + ν, [рад]")

    for ax in axes.ravel():
        ax.tick_params(axis='both', which='major', labelsize=10)

    plt.show()

In [5]:
# 1) Пересекающиеся
oi_int = KepTrue(a=6_700_000.0, e=0.00100, w=20.0 * deg, i=30.0 * deg, raan=30.0 * deg, nu=0.0)
ot_int = Kep(a=6_700_500.0, e=0.00300, w=60.0 * deg, i=30.0 * deg, raan=30.0 * deg)
plot_transition_series(oi_int, ot_int, mu)

In [30]:
# 2) Непересекающиеся
oi_non = KepTrue(a=6_700_000.0, e=0.00100, w=20.0 * deg, i=30.0 * deg, raan=30.0 * deg, nu=0.0)
ot_non = Kep(a=6_750_000.0, e=0.00120, w=30.0 * deg, i=30.0 * deg, raan=30.0 * deg)
plot_transition_series(oi_non, ot_non, mu)

In [31]:
# 3) Касающиеся (граница)
oi_tan = KepTrue(a=6_700_000.0, e=0.00100, w=0.0 * deg, i=30.0 * deg, raan=30.0 * deg, nu=0.0)
ot_tan = Kep(a=6_716_770.963704631, e=0.00150, w=180.0 * deg, i=30.0 * deg, raan=30.0 * deg)
plot_transition_series(oi_tan, ot_tan, mu)

### Зависимость хар. скорости от начального угла для случая непересекающихся орбит

In [46]:
oi = Kep(a=6_566_000.0, e=0.00228, w=20 * deg, i=0.0, raan=130 * deg)
ot = Kep(a=6_721_000.0, e=0.00149, w=150 * deg, i=0.0, raan=130 * deg)

# Параметры задачи перехода
devs = trans_devs(oi, ot)
ref = reference_orbit(oi, ot, mu)  # ref.v — круговая скорость на опорной орбите

# Сканируем угол первого импульса: ang1 ∈ [0, 2π)
thetas = np.linspace(0.0, 2.0 * np.pi, 720, endpoint=False)
dV = np.empty_like(thetas)

for k, ang1 in enumerate(thetas):
    try:
        mans = coplanar_non_intersecting(devs, ang1=ang1)  # dv безразмерные (по v_ref)
        # в непересекающемся случае импульсы чисто трансверсальные: берём y-компоненту
        dvt_sum_abs = abs(mans[0].dv[1]) + abs(mans[1].dv[1])
        dV[k] = ref.v * dvt_sum_abs
    except ZeroDivisionError:
        dV[k] = np.nan  # теоретически, когда знаменатель в формуле обнуляется

# Эталон: ΔV = v_ref * |ΔA| / 2 (не зависит от угла)
dV_ref = ref.v * abs(devs.a) / 2.0

# График
fig, ax = plt.subplots(figsize=(10, 7))
ax.axhline(dV_ref, ls='--', color='C1', label=r'эталон $v_{\rm ref}\,|\Delta A|/2$')
ax.plot(thetas, dV, lw=2, color='C0', label=r'$\Delta V(\varphi_1)$ (расчёт)')

ax.set_xlabel(r'$\varphi_1$ (рад)', fontsize=22)  # угол первого импульса
ax.set_ylabel(r'$\Delta V$ (м/с)', fontsize=22)
ax.set_title('Непересекающиеся орбиты: зависимость суммарной ΔV от угла первого импульса', fontsize=22)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=22)
plt.show()

### Годограф базис-вектора

In [47]:
def primer_coeffs(m1: Maneuver, m2: Maneuver) -> tuple[Scalar, Scalar, Scalar]:
    """
    Вычисляет коэффициенты базис-вектора для компланарного решения по двум манёврам.

    Модель (линейная, базис-вектор):
        λ(θ) = R · sin(θ − θ0)
        μ(θ) = 2·λ1 + 2·R · cos(θ − θ0)

    где
        θ0  — угол первого импульса (истинная широта точки приложения), [рад];
        s_k = sign(ΔV_tk), k=1,2  (берём знак тангенциальной компоненты импульса);
        R   = (s1 - s2) / 2.0 / (1.0 - cos(theta2 - theta1)),
        λ1  = s1 / 2 - R.

    Параметры
    ----------
    m1, m2 : Maneuver
        Два манёвра компланарного решения.

    Возвращает
    -------
    (theta0, lam1, R) : tuple[Scalar, Scalar, Scalar]
        θ0 — опорный угол; λ1 и R — коэффициенты в формулах λ(θ), μ(θ).
    """
    theta1 = float(m1.angle)
    theta2 = float(m2.angle)
    s1 = 1.0 if float(m1.dv[1]) >= 0.0 else -1.0
    s2 = 1.0 if float(m2.dv[1]) >= 0.0 else -1.0

    # фиксируем фазу как θ0 = θ1
    theta0 = theta1
    dtheta = normalize_angle(theta2 - theta1)
    cosd = math.cos(dtheta)

    dvt1_abs = abs(float(m1.dv[1]))
    dvt2_abs = abs(float(m2.dv[1]))

    if abs(cosd - 1.0) < 1e-13 or dvt1_abs < 1e-13 or dvt2_abs < 1e-13:
        # Это случай касающихся орбит. Он тут коряво обрабатывается
        max_abs_dv = float(m1.dv[1]) if dvt1_abs >= dvt2_abs else float(m2.dv[1])
        s = 1.0 if max_abs_dv >= 0.0 else -1.0
        lam1 = s * 1.0 / 8.0
        R = -(1.0 / 2.0 - abs(lam1))
        return theta0, lam1, R
    else:
        R = (s1 - s2) / 2.0 / (1.0 - cosd)
        lam1 = (s1 / 2.0) - R
        return theta0, lam1, R

In [48]:
def primer_coplanar(m1: Maneuver, m2: Maneuver, step: Scalar = np.deg2rad(1.0)) -> tuple[
    np.ndarray, np.ndarray, np.ndarray]:
    """
    Строит зависимость компонент базис-вектора (λ, μ) от угла θ для компланарного решения.

        λ(θ) = R · sin(θ − θ0)
        μ(θ) = 2·λ1 + 2·R · cos(θ − θ0)

    Параметры
    ----------
    m1, m2 : Maneuver
        Два манёвра (как в coplanar_analytical): dv в орбитальной СК {r,t,n}, angle в радианах.
        Используется только `angle` и `dv[1]` (тангенциальная компонента).
    step : Scalar, optional
        Шаг по углу θ (рад). По умолчанию 2π/720.

    Возвращает
    -------
    theta, lam, mu : (np.ndarray, np.ndarray, np.ndarray)
        Сетка углов θ ∈ [0, 2π], и соответствующие значения λ(θ), μ(θ).
    """
    theta0, lam1, R = primer_coeffs(m1, m2)

    # сетка углов [0, 2π] с включением правой границы
    n = int(math.ceil((2.0 * math.pi) / step)) + 1
    theta = np.linspace(0.0, 2.0 * math.pi, n, endpoint=True)

    d = theta - theta0
    lam = R * np.sin(d)
    mu = 2.0 * lam1 + 2.0 * R * np.cos(d)

    return theta, lam.astype(np.float64), mu.astype(np.float64)

In [49]:
def plot_primer(oi: Kep, ot: Kep, mu: float):
    """
    Слева: ΔV-«стебли» и |p(θ)|; справа: годограф (μ по горизонтали, λ по вертикали)
    с единичной окружностью и отметками точек импульсов.
    """
    mans = coplanar_analytical(oi, ot, mu)
    assert len(mans) == 2, "Ожидались два импульса в компланарном решении."

    # кривые базис-вектора
    theta, lam, muv = primer_coplanar(mans[0], mans[1])  # lam=λ(θ), muv=μ(θ)
    s_mag = np.sqrt(lam * lam + muv * muv)

    # импульсы: углы и |ΔV_t|
    impulses = [(float(mans[0].angle), float(mans[0].dv[1])),
                (float(mans[1].angle), float(mans[1].dv[1]))]
    th_imp = np.array([p for p, _ in impulses], dtype=float)
    dv_imp = np.array([v for _, v in impulses], dtype=float)

    # фигура и оси; у |p(θ)| та же горизонталь, что и у ΔV (sharex)
    fig = plt.figure(figsize=(12, 6))
    gs = fig.add_gridspec(2, 2, width_ratios=[2, 1], height_ratios=[1, 2],
                          wspace=0.25, hspace=0.25)
    ax1 = fig.add_subplot(gs[0, 0])  # ΔV
    ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)  # |p(θ)| (общая ось θ с ax1)
    ax3 = fig.add_subplot(gs[:, 1])  # годограф (μ по X, λ по Y)

    # === (1) ΔV-«стебли»
    markerline, stemlines, _ = ax1.stem(th_imp, dv_imp, basefmt=" ")
    plt.setp(stemlines, linewidth=2)
    plt.setp(markerline, markersize=6)
    ax1.set_xlim(0.0, 2.0 * np.pi)  # согласуем диапазон θ
    ax1.set_ylabel("ΔV (м/с)", fontsize=22)
    ax1.set_title("Хар. скорость", fontsize=22)
    ax1.tick_params(axis='both', which='major', labelsize=18)
    ax1.grid(True, alpha=0.35)

    # === (2) |p(θ)|
    ax2.plot(theta, s_mag, lw=2)
    ax2.axhline(1.0, color="k", ls="--", lw=2.0, alpha=0.8)
    for th in th_imp:
        ax2.axvline(th, color="r", ls=":", lw=2.0, alpha=0.8)
    ax2.set_ylim(0.0, max(1.2, float(s_mag.max()) * 1.05))
    ax2.set_ylabel("|p(θ)|", fontsize=22)
    ax2.set_xlabel("θ (рад)", fontsize=22)
    ax2.set_title("Модуль базис-вектора", fontsize=22)
    ax2.tick_params(axis='both', which='major', labelsize=18)
    ax2.grid(True, alpha=0.35)

    # === (3) Годограф
    ax3.plot(muv, lam, lw=2, label="p(θ)")
    ang = np.linspace(0, 2 * np.pi, 400)
    ax3.plot(np.sin(ang), np.cos(ang), 'k--', lw=2, alpha=0.8, label="|p|=1")  # (μ, λ)

    # точки импульсов на годографе: восстановим координаты (λ, μ) и отрисуем как (μ, λ)
    th1 = th_imp[0]
    idx1 = int(np.argmin(np.abs(theta - th1)))
    idx2 = int(np.argmin(np.abs(((theta - (th1 + np.pi)) + 2 * np.pi) % (2 * np.pi))))
    # μ(th1)=2(λ1+R), μ(th1+π)=2(λ1−R) → оценки λ1 и R
    mu1, mu2 = float(muv[idx1]), float(muv[idx2])
    R_est, l1_est = 0.25 * (mu1 - mu2), 0.25 * (mu1 + mu2)

    mult = -1

    for th in th_imp:
        d = th - th1
        lam_pt = R_est * math.sin(d)
        mu_pt = 2.0 * l1_est + 2.0 * R_est * math.cos(d)
        ax3.plot(mu_pt, lam_pt, 'ro')  # (μ, λ)
        mult *= -1
        ax3.annotate(f"θ={th:.2f}", (mu_pt, lam_pt),
                     textcoords="offset points", xytext=(6, mult * 6),
                     fontsize=14, color='r')

    ax3.set_aspect('equal', 'box')
    ax3.set_xlabel("μ (тангенциальная)", fontsize=22)
    ax3.set_ylabel("λ (радиальная)", fontsize=22)
    ax3.set_title("Годограф базис-вектора", fontsize=22)
    ax3.tick_params(axis='both', which='major', labelsize=18)
    ax3.grid(True, alpha=0.35)
    ax3.legend(loc="best", fontsize=14, framealpha=0.6)

    plt.show()

In [50]:
# 1) Пересекающиеся
oi_int = Kep(a=6_700_000.0, e=0.00100, w=20.0 * deg, i=0.0, raan=0.0)
ot_int = Kep(a=6_700_500.0, e=0.00300, w=60.0 * deg, i=0.0, raan=0.0)
plot_primer(oi_int, ot_int, mu)

In [51]:
# 2) Непересекающиеся
oi_non = Kep(a=6_700_000.0, e=0.00100, w=20.0 * deg, i=0.0, raan=0.0)
ot_non = Kep(a=6_750_000.0, e=0.00120, w=30.0 * deg, i=0.0, raan=0.0)
plot_primer(oi_non, ot_non, mu)

In [52]:
# 3) Касающиеся (граница)
oi_tan = Kep(a=6_700_000.0, e=0.00100, w=0.0 * deg, i=0.0, raan=0.0)
ot_tan = Kep(a=6_716_770.963704631, e=0.00150, w=180.0 * deg, i=0.0, raan=0.0)
plot_primer(oi_tan, ot_tan, mu)